In [2]:
import sys
%store -r SYSPATHS
sys.path.append(SYSPATHS['tool'])
import Khcore as kc
import Khcore1 as kc1
import MyEcharts as me
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from imp import reload

reload(kc)
reload(kc1)

<module 'Khcore1' from '/home/khpython/Root/tool/Khcore1.py'>

## 浓缩段温度异常工况故障树梳理

<table>
	<tr>
    <th>异常工况</th>
    <th>故障现象</th>
    <th>故障一级原因</th>
    <th>故障二级原因</th>
    <th>备注</th>
   </tr>

  <tr>
    <td rowspan="7" valign="middle" align="middle">浓缩段温度超标</td>
    <td rowspan="7" valign="middle" align="middle">浓缩段温度升高</td>
   </tr>
   
  <tr>
    <td valign="middle" align="middle">入口负荷增加</td>
    <td valign="middle" align="middle">原烟气温度升高</td>
    <td valign="middle" align="middle">对应测点 原烟气温度</td>
   <tr>
       
   <tr>
    <td valign="middle" align="middle">处于两个浓缩泵切换过程</td>
    <td> </td>
    <td valign="middle" align="middle">对应测点 浓缩泵A电流<br>浓缩泵B电流</td>
   </tr>
   
   <tr>
    <td valign="middle" align="middle">脱硫塔固含量升高</td>
    <td valign="middle" align="middle">处于出料期间</td>
    <td valign="middle" align="middle">对应测点 蒸汽母管流量<br>
塔底浆液液位<br>
固含量可以通过检测化验方式获取</td>
   </tr>
   
   <tr>
    <td valign="middle" align="middle">入口烟气含高温水</td>
    <td valign="middle" align="middle">锅炉爆管导致</td>
    <td valign="middle" align="middle">对应锅炉段测点</td>
   </tr>
</table>

# 6月份数据训练

### 2#浓缩段温度A,B,C 原始数据 时序图

In [13]:
names = ["TSIA-1204A","TSIA-1204B","TSIA-1204C"]
datas=  []
for name in names:
    datas.append(kc1.ReadRawData(name))
data = pd.concat(datas,axis=1)
data= data[data.index>"2019-06-13 19:00:00"]   ## 2019-06-13 19:00:00 之前属于数据测点调试阶段
data.columns = names
data1 = data.resample("1min").mean()
me.Plot_LineBar(data1,name="TSIA-1204_06_raw",overwrite=True)

### 根据浓缩段温度时序图 对数据打标签
    浓缩段温度A 传感器对温度敏感, 其变化幅度较大的区域或者 温度绝对值较高的 区域认为是 浓缩段温度异常区域

In [4]:
bins = [["2019-06-14 13:20:00","2019-06-14 17:28:00"],["2019-06-16 05:05:00","2019-06-16 15:40:00"],
       ["2019-06-17 05:00:00","2019-06-17 17:31:00"],["2019-06-18 11:08:00","2019-06-18 17:36:00"],\
       ["2019-06-22 07:21:00","2019-06-22 15:51:00"],["2019-06-23 08:30:00","2019-06-23 17:00:00"],\
        ["2019-06-24 23:40:00","2019-06-25 03:34:00"],["2019-06-25 23:30:00","2019-06-26 00:35:00"],\
        ["2019-06-27 06:55:00","2019-06-27 07:57:00"],["2019-06-27 13:32:00","2019-06-28 04:19:00"],\
        ["2019-06-28 15:12:00","2019-06-29 06:35:00"],["2019-06-30 02:31:00","2019-06-30 07:30:00"],\
       ]
labels = [1]*len(bins)

### 设置计算步进为 1min, 即在线系统一分钟触发一次计算
    data1 即为 历史数据data 的 1min降采样数据, 即以此数据作为训练数据，打数据标签
    数据窗口长度为 5min

In [7]:
step = "1min" 
window ="5min"
data2 = kc1.generate_label(data1,bins,labels)
data2['label'] =data2['label'].fillna(0)

### 加工特征：
    1. 当前时刻平均温度 : 浓缩段温度A,B,C 去掉最大值, 剩余两个温度的均值
    2. 当前时刻温度差: 浓缩段温度A,B,C 的最大值减去平均温度
    3. 当前时刻向前滑动窗口下的平均温度斜率 : 滑动窗口取 5min , 即前5个平均温度的拟合斜率

In [8]:
## 1. 浓缩段平均温度变量
df = kc1.ReadRawData("TE_1003")  ## 根据 kc1.GenerateAvgTemp 计算的 平均温度
df1 = df[df.index>"2019-06-13 19:00:00"]
data2["TE_1003"] = df1['value']
data2= data2.dropna()

## 2. 温度峰峰值
data2['diff'] = data2[['TE_1003a','TE_1003b','TE_1003c']].max(axis=1)-data2['TE_1003']

## 3. 温度斜率
data2["slope"] = data2['TE_1003'].rolling(window=window).apply(kc1.gradient)
data2= data2.dropna()

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # This is added back by InteractiveShellApp.init_path()


In [11]:
data2.head()

,TE_1003a,TE_1003b,TE_1003c,label,TE_1003,diff,slope
time,,,,,,,
2019-06-13 19:01:00,61.558003,59.875460,59.851057,0.0,58.38830,3.169703,0.000000
2019-06-13 19:02:00,61.684993,59.822970,59.844953,0.0,57.91216,3.772833,-0.476140
2019-06-13 19:03:00,61.789990,59.888887,59.896207,0.0,57.99264,3.797350,-0.197830
2019-06-13 19:04:00,61.796087,59.901080,59.914513,0.0,58.08054,3.715547,-0.084280
2019-06-13 19:05:00,61.678883,59.672787,59.102577,0.0,57.88282,3.796063,-0.084258


### 特征数据标准化

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

X= data2[['TE_1003','diff','slope']]
Y = data2['label']
ss = StandardScaler() ## 数据标准化
ss.fit(X)
X1 = ss.transform(X)
pd.DataFrame(X1,columns = ['TE_1003','diff','slope'],index=X.index).head() ## 标准化数据

,TE_1003,diff,slope
time,,,
2019-06-13 19:01:00,0.003843,0.753110,-0.000131
2019-06-13 19:02:00,-0.187747,1.079130,-1.393231
2019-06-13 19:03:00,-0.155363,1.092382,-0.578946
2019-06-13 19:04:00,-0.119994,1.048164,-0.246719
2019-06-13 19:05:00,-0.199553,1.091687,-0.246654


### 模型训练 - Lasso

In [42]:
## 采用 Lasso 的回归模型
from sklearn import linear_model
model = linear_model.Lasso(alpha=0.05) 
model.fit(X1,Y)
print (model.coef_)
data2['Lasso'] = model.predict(X1)
me.Plot_LineBar(data2,columns=['label','Lasso'],name="TE_1003_06_Lasso",overwrite=True)
### slope 含有 连续时序特征, 可根据Lasso 预测值 作为报警概率设定值, 默认为0.5

[ 0.1242725   0.16926137 -0.        ]


In [43]:
## 采用 LinearRegression 的回归模型
from sklearn import linear_model
model1 = linear_model.LinearRegression()
model1.fit(X1,Y)
print (model1.coef_)
data2['Linear'] = model1.predict(X1)
me.Plot_LineBar(data2,columns=['label','Linear'],name="TE_1003_06_Linear",overwrite=True)

[ 0.18757425  0.2297963  -0.00737022]


### 两模型及原始数据对比

In [51]:
data2['TE_1003_scale'] = X1[:,0]
me.Plot_LineBar(data2,columns=['TE_1003_scale','label','Lasso','Linear'],\
                name="TE_1003_06_predict",overwrite=True)

# 7月份数据测试

### 加载原始数据

In [14]:
names = ["TSIA-1204A","TSIA-1204B","TSIA-1204C"]
datas=  []
for name in names:
    datas.append(kc1.ReadRawData(name,month="2019_07"))
data07 = pd.concat(datas,axis=1)
data07.columns = names
data1_07 = data07.resample("1min").mean()
me.Plot_LineBar(data1_07,name="TSIA-1204_07_raw",overwrite=True)

### 加工数据特征

In [39]:
## 1. 浓缩段平均温度变量
# df_07=kc1.GenerateAvgTemp(["TE_1003a","TE_1003b","TE_1003c"],month="2019_07")
df_07 = kc1.ReadRawData("TE_1003",month="2019_07")
data1_07["TE_1003"] = df_07['value']
data1_07= data1_07.dropna()

## 2. 温度峰峰值
data1_07['diff'] = data1_07[['TE_1003a','TE_1003b','TE_1003c']].max(axis=1)-data1_07['TE_1003']

## 3. 温度斜率
data1_07["slope"] = data1_07['TE_1003'].rolling(window=window).apply(kc1.gradient)
data1_07= data1_07.dropna()

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # This is added back by InteractiveShellApp.init_path()


In [40]:
data1_07.head()

,TE_1003a,TE_1003b,TE_1003c,TE_1003,diff,slope
time,,,,,,
2019-07-01 00:00:00,60.357757,60.599510,60.175827,59.83144,0.768070,0.000000
2019-07-01 00:01:00,60.622693,60.810750,60.496937,60.19046,0.620290,0.359020
2019-07-01 00:02:00,60.914530,60.923087,60.517693,60.34432,0.578767,0.256440
2019-07-01 00:03:00,61.050080,60.970697,60.705730,60.64466,0.405420,0.259352
2019-07-01 00:04:00,61.148950,61.081793,60.802213,60.74722,0.401730,0.228576


In [41]:
X_07= data1_07[['TE_1003','diff','slope']]
X1_07 = ss.transform(X_07)

### 模型效果验证

In [50]:
data1_07['Lasso'] = model.predict(X1_07)
data1_07['Linear'] = model1.predict(X1_07)
data1_07['TE_1003_scale'] = X1_07[:,0]
me.Plot_LineBar(data1_07,columns=['TE_1003_scale','Lasso','Linear'],\
                name="TE_1003_07_predict",overwrite=True)